# Proyecto

In [3]:
import numpy as np
import pandas as pd



In [4]:
dataSet = np.load('proyecto_training_data.npy')

In [23]:
dataP = dataSet[:int(len(dataSet)*0.8)]
dataA = dataSet[int(len(dataSet)*0.8):]
print(len(dataSet))
print(len(dataP))
print(len(dataA))
df = pd.DataFrame(dataP, columns = ['X0','X1','X2','X3','X4','X5'])
print(df)

1460
1168
292
            X0   X1      X2   X3      X4    X5
0     208500.0  7.0   856.0  8.0  2003.0  65.0
1     181500.0  6.0  1262.0  6.0  1976.0  80.0
2     223500.0  7.0   920.0  6.0  2001.0  68.0
3     140000.0  7.0   961.0  7.0  1915.0  60.0
4     250000.0  8.0  1145.0  9.0  2000.0  84.0
...        ...  ...     ...  ...     ...   ...
1163  108959.0  4.0  1258.0  6.0  1969.0  60.0
1164  194000.0  5.0  1432.0  5.0  1978.0   NaN
1165  233170.0  7.0  1502.0  7.0  2009.0  79.0
1166  245350.0  8.0  1694.0  7.0  2008.0  64.0
1167  173000.0  6.0   959.0  7.0  2000.0  58.0

[1168 rows x 6 columns]


**Media de cada variable:**

In [32]:
#print(df.iloc[:,[0]])
medianX0 = np.mean(df.iloc[:,[0]])
medianX1 = np.mean(df.iloc[:,[1]])
medianX2 = np.mean(df.iloc[:,[2]])
medianX3 = np.mean(df.iloc[:,[3]])
medianX4 = np.mean(df.iloc[:,[4]])
medianX5 = np.mean(df.iloc[:,[5]])
print(medianX0,medianX1,medianX2,medianX3,medianX4,medianX5)

X0    180590.277397
dtype: float64 X1    6.101027
dtype: float64 X2    1156.32363
dtype: float64 X3    6.485445
dtype: float64 X4    1971.451199
dtype: float64 X5    69.946985
dtype: float64
